In [13]:
RANDOM_STATE = 101
VERSION = '11'

no_dummies = False # 'no dummies' in DATA_DETAIL
no_dummies = True

AUTOENCODE_EPOCHS = 400
AUTOENCODE_EPOCHS = 80
#AUTOENCODE_EPOCHS = 10

!pip install graphviz
!pip install pydot

In [14]:
import sys
import os
import json

module_path = os.path.abspath(os.path.join('..', '..', '..'))
if module_path not in sys.path:
    #sys.path.append(module_path+"\\zfunctions")
    sys.path.append(module_path)

#with open('../../z_envs/_envs.json') as f:
#    env_vars = json.loads(f.read())

try:
    import google.colab

    run_env = 'colab'
except:
    try:
        run_env = env_vars['notebook_environment']
    except:
        run_env = 'unknown'

if "JPY_PARENT_PID" in os.environ:
    is_jupyter = True
else:
    is_jupyter = False



if run_env not in ['colab', 'gradient', 'cloud','unknown']:
    cloud_run = False
    from functions_b__get_the_data_20221116 import set_csv_directory
    set_csv_directory('final_split')
else:
    cloud_run = True

from functions_0__common_20221116 import get_columns
from functions_b__get_the_data_20221116 import get_combined_dataset, get_source_dataframe
from functions_d1__prepare_cleanse_data_20221116 import tidy_dataset
from functions_d2__transform_enrich_data_20221116 import preprocess, feature_engineer
from functions_d3__prepare_store_data_20221116 import create_train_test_data
from functions_e__train_model_20221116 import get_chosen_model, make_modelling_pipeline, get_cv_params, fit_model_with_cross_validation, get_hyperparameters
from functions_f_evaluate_model_20221116 import get_best_estimator_average_time, get_results, update_results

#print(env_vars)

In [15]:
columns, booleans, floats, categories, custom, wildcard = get_columns(version=VERSION)
LABEL = 'Price'

In [16]:
df, retrieval_type = get_source_dataframe(cloud_run, VERSION, folder_prefix='../../../', row_limit=None)
df_orig = df.copy()

if retrieval_type != 'tidy':
    df = tidy_dataset(df, version=int(VERSION))
    df = feature_engineer(df, version=int(VERSION))


    df = df[columns]

loaded data from ../../../https://raw.githubusercontent.com/jayportfolio/capstone_streamlit/main/data/final/df_listings_v11.csv


In [17]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig, X_train_index, X_test_index, y_train_index, y_test_index, df_features, df_labels = create_train_test_data(
    df,
    categories=categories,
    RANDOM_STATE=RANDOM_STATE, return_index=True,
    drop_nulls=True,
    no_dummies=no_dummies
)

#print(X_train[0])
print(df.shape)
print(X_train_orig.shape, X_test_orig.shape, y_train_orig.shape, y_test_orig.shape, X_train_index.shape, X_test_index.shape,
      y_train_index.shape, y_test_index.shape)



(46871, 81)
(39714, 80) (4413, 80) (39714, 1) (4413, 1) (39714, 1) (4413, 1) (39714, 1) (4413, 1)


In [18]:
X_train = X_train_orig.copy()
X_test = X_test_orig.copy()

In [19]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Model, load_model # for creating a Neural Network Autoencoder model
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, LeakyReLU, BatchNormalization # for adding layers to AE model
from tensorflow.keras.utils import plot_model # for plotting model diagram

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # For rescaling metrics to fit into 0 to 1 range
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples

# Visualization
import matplotlib
import matplotlib.pyplot as plt # for plotting model loss
print('matplotlib: %s' % matplotlib.__version__) # print version
import graphviz # for showing model diagram
print('graphviz: %s' % graphviz.__version__) # print version

# Other utilities
import sys
import os

# Assign main directory to a variable
main_dir=os.path.dirname(sys.path[0])
#print(main_dir)

Tensorflow/Keras: 2.9.0
pandas: 1.4.3
sklearn: 1.1.1
matplotlib: 3.5.2
graphviz: 0.20.1


In [20]:
#--- Define Shapes
n_inputs=X_train.shape[1] # number of input neurons = the number of features in X_train
n_bottleneck=(round(float(n_inputs) / 2.0)) # bottleneck to have half the number of neurons

#--- Input Layer
visible = Input(shape=(n_inputs,), name='Input-Layer') # Specify input shape

#--- Encoder Layer
e = Dense(units=n_inputs, name='Encoder-Layer')(visible)
e = BatchNormalization(name='Encoder-Layer-Normalization')(e)
e = LeakyReLU(name='Encoder-Layer-Activation')(e)

#--- Bottleneck
bottleneck = Dense(units=n_bottleneck, name='Bottleneck-Layer')(e)

#--- Decoder Layer
d = Dense(units=n_inputs, name='Decoder-Layer')(bottleneck)
d = BatchNormalization(name='Decoder-Layer-Normalization')(d)
d = LeakyReLU(name='Decoder-Layer-Activation')(d)

#--- Output layer
output = Dense(units=n_inputs, activation='linear', name='Output-Layer')(d)

# Define autoencoder model
model = Model(inputs=visible, outputs=output, name='Autoencoder-Model')

# Compile autoencoder model
model.compile(optimizer='adam', loss='mse')

# Print model summary
print(model.summary())


# Plot the autoencoder model diagram
try:
    plot_model(model, to_file=main_dir+'/pics/Autoencoder.png', show_shapes=True, dpi=300)
except:
    plot_model(model, to_file='./pics/Autoencoder.png', show_shapes=True, dpi=300)


Model: "Autoencoder-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input-Layer (InputLayer)    [(None, 80)]              0         
                                                                 
 Encoder-Layer (Dense)       (None, 80)                6480      
                                                                 
 Encoder-Layer-Normalization  (None, 80)               320       
  (BatchNormalization)                                           
                                                                 
 Encoder-Layer-Activation (L  (None, 80)               0         
 eakyReLU)                                                       
                                                                 
 Bottleneck-Layer (Dense)    (None, 40)                3240      
                                                                 
 Decoder-Layer (Dense)       (None, 80)          

In [21]:
# Fit the autoencoder model to reconstruct input
history = model.fit(X_train, X_train, epochs=50, batch_size=16, verbose=1, validation_data=(X_test, X_test))

# Plot a loss chart
fig, ax = plt.subplots(figsize=(16,9), dpi=300)
plt.title(label='Model Loss by Epoch', loc='center')

ax.plot(history.history['loss'], label='Training Data', color='black')
ax.plot(history.history['val_loss'], label='Test Data', color='red')
ax.set(xlabel='Epoch', ylabel='Loss')
plt.legend()

plt.show()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
suffix = '_no_dummies' if no_dummies else ''
suffix

In [10]:
# Define an encoder model without the decoder
encoder = Model(inputs=visible, outputs=bottleneck)

# Compile encoder model
encoder.compile(optimizer='adam', loss='mse')


# Save the encoder model to file
try:
    main_dir=os.path.dirname(sys.path[0])
    encoder.save(main_dir+f'/data/encoder_v{VERSION}{suffix}.h5')
    print('performed operation to main_dir: SAVE')
except:
    print('failed to perform operation to main_dir: SAVE')
    main_dir = '.'
    encoder.save(main_dir+f'/data/encoder_v{VERSION}.h5')
    print('performed operation to local/relative: SAVE')

# Plot the autoencoder model diagram
plot_model(encoder, to_file=main_dir+'/pics/Encoder_only.png', show_shapes=True, dpi=300)

performed operation to main_dir: SAVE
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [11]:
# Load the model from file
encoder = load_model(main_dir+f'/data/encoder_v{VERSION}.h5')

# Encode train and test data
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

138/138 [==============================] - 0s 1ms/step


In [64]:
X_train_orig[0:3]


array([[ 3.00000000e+00,  3.00000000e+00,  3.16613257e-01,
         5.14597600e+01,  1.29390000e-01,  3.99600000e-02,
         2.33810000e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+0

In [65]:
X_train[0:3]

array([[ 3.00000000e+00,  3.00000000e+00,  3.16613257e-01,
         5.14597600e+01,  1.29390000e-01,  3.99600000e-02,
         2.33810000e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+0

In [66]:
X_train_encoded[0:3]

array([[  6.614075  , -10.00087   ,   2.0106304 ,  10.822067  ,
         -8.197215  ,   0.0887069 ,  -4.333485  ,  -1.5984313 ,
         -6.916433  , -10.046262  ,  -7.6852107 ,   2.119754  ,
         -8.780337  ,  -2.89051   ,   3.41351   ,   3.6929772 ,
          5.875437  ,   3.61391   ,  -1.549049  ,  -3.780601  ,
         -1.7301445 ,   9.872017  ,  -2.232491  ,  -4.7083654 ,
         -4.614657  ,  -6.096317  ,   2.7854896 ,   6.265664  ,
          9.170942  ,   4.5868783 ,   0.7995242 ,   6.702379  ,
          1.5992277 ,   4.859478  ,   1.641658  ,   5.9614625 ,
          2.6014006 ,  -0.5302769 ,  -5.540656  ,   8.198903  ,
          5.8713994 ,  -6.3346806 ],
       [  6.452407  ,  -7.552716  ,   0.8782065 ,   9.054948  ,
         -8.954256  ,  -0.715046  ,  -6.8363    ,   1.9834826 ,
         -7.8510723 ,  -4.2933545 ,  -3.4465954 ,   7.1777143 ,
         -8.305194  ,  -6.9156885 ,   2.9866521 ,   2.400256  ,
         -0.0485765 ,  -0.716206  ,   0.1656414 ,  -3.6421177 ,
   

In [67]:
y_train = y_train_orig
y_test = y_test_orig

In [68]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor

# define model
#model = SVR()
#model = LinearRegression()
model = KNeighborsRegressor()

# fit model on the training dataset
model.fit(X_train_encoded, y_train)
# make prediction on test set

KNeighborsRegressor()

In [69]:
y_pred = model.predict(X_test_encoded)
y_pred_orig = y_pred

In [70]:
# invert transforms so we can calculate errors
y_pred = y_pred.reshape((len(y_pred), 1))


#trans_out = MinMaxScaler()
#trans_out.fit(y_train)

#y_pred = trans_out.inverse_transform(y_pred)
#??? y_test = trans_out.inverse_transform(y_test)
# calculate error

In [71]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
mae_score = mean_absolute_error(y_test, y_pred)
score = r2_score(y_test, y_pred)
print(mae_score)
print(score)

68165.35041921595
0.34463715278837204


In [72]:
print(y_test[0], y_pred[0])


[400000.] [358100.]


In [73]:
import numpy as np

print("Score (mean absolute error):", mean_absolute_error(y_test, y_pred), "\n")
compare_np = np.hstack((y_test, y_pred))
compare_df = pd.DataFrame(compare_np, columns=['actual', 'predicted'])
compare_df['diff 1 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['actual'] * 100)
compare_df['diff 2 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['predicted']) * 100

compare_df

Score (mean absolute error): 68165.35041921595 



,actual,predicted,diff 1 %,diff 2 %
0,400000.0,358100.0,10.475000,11.700642
1,369950.0,374000.0,1.094743,1.082888
2,350000.0,296000.0,15.428571,18.243243
3,549000.0,528590.0,3.717668,3.861216
4,495000.0,436000.0,11.919192,13.532110
...,...,...,...,...
4408,475000.0,472000.0,0.631579,0.635593
4409,575000.0,555990.0,3.306087,3.419126
4410,400000.0,340000.0,15.000000,17.647059
4411,450000.0,505000.0,12.222222,10.891089


In [ ]:
import matplotlib.pyplot as plt
best_model_fig, best_model_ax = plt.subplots()
best_model_ax.scatter(y_test, y_pred, edgecolors=(0, 0, 1))
best_model_ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
best_model_ax.set_ylabel('Predicted')
best_model_ax.set_xlabel('Actual')
#ax.title.set_text(f'CV Chosen best option ({calculated_best_pipe[1]})')

plt.show()